In [ ]:
!git clone https://github.com/Team-M1/badwords-classifier-train
%cd badwords-classifier-train
!pip install -r requirements.txt

# 모델 불러오기

각자 알아서 불러올 것

In [2]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer


model = AutoModelForSequenceClassification.from_pretrained("대충 모델")
tokenizer = AutoTokenizer.from_pretrained("대충 토크나이저")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'ElectraTokenizer'. 
The class this function is called from is 'KoCharElectraTokenizer'.


In [17]:
import numpy as np
import pandas as pd


test_df = pd.read_csv("data/test.csv")
test_data = test_df["content"]
test_label = test_df["labels"]

speed_test_data = test_data.head(100)

In [21]:
model.to("cpu")
model.eval()
print()

## 스피드 테스트

In [5]:
# test
%%timeit

with torch.no_grad():
    tokens = tokenizer(list(speed_test_data), padding="max_length", truncation=True, return_tensors="pt")
    _ = model(**tokens)

## F1 스코어

In [ ]:
# f1
model.cuda()

with torch.no_grad():
    tokens = tokenizer(list(test_data), padding="max_length", truncation=True, return_tensors="pt")
    tokens = tokens.to("cuda")
    preds = model(**tokens)

In [ ]:
from sklearn.metrics import f1_score


logits = preds.logits.cpu().numpy()
pred = np.argmax(logits, 1)

f1 = f1_score(test_label, pred, average="macro")
print(f1)